# Predicting Subscription to a Video Game Newsletter Based on Age and Time Played on the Game

The question we aim to answer through this report is *'Can age and time played of a player predict subscription to a newsletter in players.csv?'*

The dataset titled "players.csv" will be used to answer this question. The dataset was produced by collecting data from the players of a video game called 'Minecraft'. The information included is: 
 - The players experience
 - Whether or not the player has suscribed to a game-related newsletter
 - The hashed email of the player
 - The hours they have spent playing
 - The name of the player
 - The gender of the player
 - The age of the player

For this report we will be focusing on just the age of the player, the time that they have spent playing, and whether or not they have suscribed to a game-related newsletter.